In [1]:
import os
import cdsw
from src.cml_extensions.dask_cluster.dask_cluster import DaskCluster

#os.environ[""]

cluster = DaskCluster(num_workers=10, worker_cpu=60, worker_memory=360, scheduler_cpu=160, scheduler_memory=360)
cluster.init()


--------------------
Dask cluster started
--------------------

The Dask dashboard is running at
https://vyhxngjkkxcroo27.ml-f0942348-224.paul-aug.a465-9q4k.cloudera.site/status

To connect to this Dask cluster from this CML Session,
use the following Python code:
  from dask.distributed import Client
  client = Client('tcp://100.100.149.214:8786')



In [2]:
from multiprocessing import cpu_count
import time

In [3]:
cpu_count()

192

In [4]:
from dask.distributed import Client
client = Client('tcp://100.100.149.214:8786')

client

<Client: 'tcp://100.100.149.214:8786' processes=10 threads=1920, memory=3.27 TiB>

## Read Parquet from S3

In [43]:
import os
import dask.dataframe as dd

df_left = dd.read_parquet("s3://paul-aug26-buk-a3c2b50a/data/pdefusco/daskdist/1kcols_10krows_100parts_wid")
df_left.head()

,unique_id,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col990,col991,col992,col993,col994,col995,col996,col997,col998,col999
0,ID_5571,A,2155749.0,2162552.0,8879422.0,3592760.0,6260197.0,4380193.0,649749.0,8567364.0,...,9557426.0,2939619.0,9470527.0,6685843.0,7080629.0,624918.0,6043804.0,864526.0,9507048.0,9039809.0
1,ID_3471,B,1461406.0,6489945.0,1028377.0,252657.0,7100934.0,1732775.0,9956641.0,3115993.0,...,228696.0,2654007.0,3404902.0,6831505.0,6674835.0,4216251.0,8204066.0,2112541.0,5544509.0,5233644.0
2,ID_4265,C,1327473.0,8855601.0,9027891.0,6230956.0,3983144.0,1449433.0,3296800.0,4257279.0,...,2704330.0,3296694.0,1627670.0,5813634.0,713084.0,7534500.0,3773338.0,1920029.0,6656024.0,309869.0
3,ID_1628,D,4750508.0,7597960.0,6513623.0,2545998.0,252819.0,8349154.0,8894517.0,6153052.0,...,4724775.0,1643458.0,2467381.0,8839086.0,4056600.0,9672412.0,7928867.0,3507550.0,3918131.0,9253725.0
4,ID_4452,E,7268317.0,6482051.0,8168993.0,7054196.0,3747121.0,6873729.0,2527835.0,1162200.0,...,8392205.0,9791940.0,2783996.0,1724811.0,8806389.0,4347406.0,1311936.0,5028528.0,4211286.0,9844447.0


In [44]:
df_right = dd.read_parquet("s3://paul-aug26-buk-a3c2b50a/data/pdefusco/daskdist/1kcols_10krows_100parts_wid")
df_right.head()

,unique_id,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col990,col991,col992,col993,col994,col995,col996,col997,col998,col999
0,ID_5571,A,2155749.0,2162552.0,8879422.0,3592760.0,6260197.0,4380193.0,649749.0,8567364.0,...,9557426.0,2939619.0,9470527.0,6685843.0,7080629.0,624918.0,6043804.0,864526.0,9507048.0,9039809.0
1,ID_3471,B,1461406.0,6489945.0,1028377.0,252657.0,7100934.0,1732775.0,9956641.0,3115993.0,...,228696.0,2654007.0,3404902.0,6831505.0,6674835.0,4216251.0,8204066.0,2112541.0,5544509.0,5233644.0
2,ID_4265,C,1327473.0,8855601.0,9027891.0,6230956.0,3983144.0,1449433.0,3296800.0,4257279.0,...,2704330.0,3296694.0,1627670.0,5813634.0,713084.0,7534500.0,3773338.0,1920029.0,6656024.0,309869.0
3,ID_1628,D,4750508.0,7597960.0,6513623.0,2545998.0,252819.0,8349154.0,8894517.0,6153052.0,...,4724775.0,1643458.0,2467381.0,8839086.0,4056600.0,9672412.0,7928867.0,3507550.0,3918131.0,9253725.0
4,ID_4452,E,7268317.0,6482051.0,8168993.0,7054196.0,3747121.0,6873729.0,2527835.0,1162200.0,...,8392205.0,9791940.0,2783996.0,1724811.0,8806389.0,4347406.0,1311936.0,5028528.0,4211286.0,9844447.0


In [45]:
#df_right

In [46]:
#value = "2155749.0"
#df_right.query(f'col2 == {value}').compute()

In [47]:
#df_list=[df_left,df_right]
#for i, df in enumerate(df_list, 1):
#    df.columns = [col_name+'_df{}'.format(i) for col_name in df.columns]

In [48]:
df_join = df_left.set_index('unique_id').join(df_right.set_index('unique_id'), how='inner', on='unique_id', lsuffix='_left2', rsuffix='_right2')  

In [49]:
df_join.head()

,col1_left2,col2_left2,col3_left2,col4_left2,col5_left2,col6_left2,col7_left2,col8_left2,col9_left2,col10_left2,...,col990_right2,col991_right2,col992_right2,col993_right2,col994_right2,col995_right2,col996_right2,col997_right2,col998_right2,col999_right2
unique_id,,,,,,,,,,,,,,,,,,,,,
ID_10,F,281301.0,8822804.0,967458.0,3293650.0,118040.0,1331221.0,8975471.0,8299249.0,4389009.0,...,6023759.0,7185839.0,70167.0,8904412.0,4581633.0,5397366.0,5975791.0,3681083.0,1887082.0,420958.0
ID_1001,B,9684350.0,7516412.0,3106392.0,1823102.0,9046900.0,4460684.0,7517061.0,7365083.0,3753966.0,...,5993710.0,6708312.0,9069300.0,8425060.0,9972092.0,2153833.0,2926195.0,8990114.0,3074770.0,7857332.0
ID_1002,D,4588598.0,9488075.0,7632077.0,3009766.0,6268436.0,8189556.0,896352.0,3457243.0,70026.0,...,3250802.0,8850967.0,5378132.0,5750603.0,1428063.0,8144155.0,9751898.0,6037035.0,4836999.0,2609419.0
ID_1006,F,6300878.0,8126670.0,1090184.0,4002889.0,6415746.0,1482758.0,7627895.0,914722.0,9833827.0,...,955847.0,8546640.0,7084117.0,2761490.0,1473434.0,1149022.0,5305717.0,3147748.0,2039563.0,8576564.0
ID_1007,F,8344172.0,6701361.0,1652607.0,5393482.0,4071646.0,8453070.0,8916602.0,84165.0,8229472.0,...,1571779.0,842545.0,137048.0,9940053.0,320665.0,7970743.0,6998732.0,3609756.0,5702410.0,2074387.0


In [50]:
df3 = dd.read_parquet("s3://paul-aug26-buk-a3c2b50a/data/pdefusco/daskdist/1kcols_10krows_100parts_wid")
df4 = dd.read_parquet("s3://paul-aug26-buk-a3c2b50a/data/pdefusco/daskdist/1kcols_10krows_100parts_wid")
df5 = dd.read_parquet("s3://paul-aug26-buk-a3c2b50a/data/pdefusco/daskdist/1kcols_10krows_100parts_wid")

In [51]:
df_join = df_join.set_index('unique_id').join(df3.set_index('unique_id'), how='inner', on='unique_id', lsuffix='_left3', rsuffix='_right3')
df_join = df_join.set_index('unique_id').join(df4.set_index('unique_id'), how='inner', on='unique_id', lsuffix='_left4', rsuffix='_right4')
df_join = df_join.set_index('unique_id').join(df5.set_index('unique_id'), how='inner', on='unique_id', lsuffix='_left5', rsuffix='_right5')

In [52]:
df_join.head()

,col1_left2,col2_left2,col3_left2,col4_left2,col5_left2,col6_left2,col7_left2,col8_left2,col9_left2,col10_left2,...,col990,col991,col992,col993,col994,col995,col996,col997,col998,col999
unique_id,,,,,,,,,,,,,,,,,,,,,
ID_10,F,281301.0,8822804.0,967458.0,3293650.0,118040.0,1331221.0,8975471.0,8299249.0,4389009.0,...,6023759.0,7185839.0,70167.0,8904412.0,4581633.0,5397366.0,5975791.0,3681083.0,1887082.0,420958.0
ID_1001,B,9684350.0,7516412.0,3106392.0,1823102.0,9046900.0,4460684.0,7517061.0,7365083.0,3753966.0,...,5993710.0,6708312.0,9069300.0,8425060.0,9972092.0,2153833.0,2926195.0,8990114.0,3074770.0,7857332.0
ID_1002,D,4588598.0,9488075.0,7632077.0,3009766.0,6268436.0,8189556.0,896352.0,3457243.0,70026.0,...,3250802.0,8850967.0,5378132.0,5750603.0,1428063.0,8144155.0,9751898.0,6037035.0,4836999.0,2609419.0
ID_1006,F,6300878.0,8126670.0,1090184.0,4002889.0,6415746.0,1482758.0,7627895.0,914722.0,9833827.0,...,955847.0,8546640.0,7084117.0,2761490.0,1473434.0,1149022.0,5305717.0,3147748.0,2039563.0,8576564.0
ID_1007,F,8344172.0,6701361.0,1652607.0,5393482.0,4071646.0,8453070.0,8916602.0,84165.0,8229472.0,...,1571779.0,842545.0,137048.0,9940053.0,320665.0,7970743.0,6998732.0,3609756.0,5702410.0,2074387.0


In [53]:
df_join = df_join.categorize('col1_left2')
df_join = df_join.reset_index(drop=False)

In [54]:
df_join.head()

,unique_id,col1_left2,col2_left2,col3_left2,col4_left2,col5_left2,col6_left2,col7_left2,col8_left2,col9_left2,...,col990,col991,col992,col993,col994,col995,col996,col997,col998,col999
0,ID_10,F,281301.0,8822804.0,967458.0,3293650.0,118040.0,1331221.0,8975471.0,8299249.0,...,6023759.0,7185839.0,70167.0,8904412.0,4581633.0,5397366.0,5975791.0,3681083.0,1887082.0,420958.0
1,ID_1001,B,9684350.0,7516412.0,3106392.0,1823102.0,9046900.0,4460684.0,7517061.0,7365083.0,...,5993710.0,6708312.0,9069300.0,8425060.0,9972092.0,2153833.0,2926195.0,8990114.0,3074770.0,7857332.0
2,ID_1002,D,4588598.0,9488075.0,7632077.0,3009766.0,6268436.0,8189556.0,896352.0,3457243.0,...,3250802.0,8850967.0,5378132.0,5750603.0,1428063.0,8144155.0,9751898.0,6037035.0,4836999.0,2609419.0
3,ID_1006,F,6300878.0,8126670.0,1090184.0,4002889.0,6415746.0,1482758.0,7627895.0,914722.0,...,955847.0,8546640.0,7084117.0,2761490.0,1473434.0,1149022.0,5305717.0,3147748.0,2039563.0,8576564.0
4,ID_1007,F,8344172.0,6701361.0,1652607.0,5393482.0,4071646.0,8453070.0,8916602.0,84165.0,...,1571779.0,842545.0,137048.0,9940053.0,320665.0,7970743.0,6998732.0,3609756.0,5702410.0,2074387.0


In [60]:
df_join.dtypes

unique_id     category
col1_left2    category
col2_left2     float32
col3_left2     float32
col4_left2     float32
                ...   
col995         float32
col996         float32
col997         float32
col998         float32
col999         float32
Length: 4996, dtype: object

In [66]:
df_pivot = dd.pivot_table(df_join, index='unique_id', columns='col1_left2', values='col2_left2', aggfunc='mean')

In [68]:
df_pivot

,A,B,C,D,E,F,G
npartitions=1,,,,,,,
,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...


In [69]:
df_pivot_flat = df_pivot.reset_index()

In [70]:
df_pivot_flat

,unique_id,A,B,C,D,E,F,G
npartitions=1,,,,,,,,
,category[known],float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...


In [71]:
df_pivot_flat.to_parquet('s3://paul-aug26-buk-a3c2b50a/data/pdefusco/daskdist/joinpivot_outputs', engine='pyarrow', overwrite=True)